In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV
from splitrepeat import splitrepeat_cv

config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

features=['moca_digits','moca_letters','moca_serial7','MMSPELL_early','MMSPELL_late','ADAS_Q1','moca_orient','ADAS_Q7','ADAS_Q8','ADAS_Q9','MMDATE',
       'MMYEAR', 'MMMONTH', 'MMDAY', 'MMSEASON', 'MMHOSPIT', 'MMFLOOR','MMCITY', 'MMAREA', 'MMSTATE','nbspan_forward',
       'faq7','faq8','moca_recall','ADAS_Q4','MMBALLDL', 'MMFLAGDL', 'MMTREEDL', 'LDELTOTAL', 'AVRECALL', 'AVDEL30MIN', 'AVDELTOT', 'AVDELERR2',
       'faq9','moca_clock','MMDRAW','CLOCKCIRC', 'CLOCKSYM','CLOCKNUM', 'CLOCKHAND', 'CLOCKTIME', 'COPYCIRC', 'COPYSYM',
       'COPYNUM', 'COPYHAND', 'COPYTIME', 'tmab_time', 'TRAAERRCOM','TRAAERROM', 'TRABERRCOM', 'TRABERROM','moca_naming','moca_repeat','moca_fluency',
       'moca_similarities','ADAS_Q2', 'ADAS_Q5','ADAS_Q10','ADAS_Q11','ADAS_Q12','MMBALL', 'MMFLAG', 'MMTREE',
       'MMWATCH', 'MMPENCIL','MMREPEAT','MMREAD', 'MMWRITE', 'CATANIMSC', 'CATVEGESC', 'moca_visuo_exec','ADAS_Q3','ADAS_Q6','ADAS_Q13','MMHAND', 'MMFOLD',
       'MMONFLR','nbspan_backward','faq1','faq2','faq3', 'faq4','faq5','faq6','faq10','PXGENAPP', 'PXHEADEY', 'PXNECK', 'PXCHEST',
       'PXHEART', 'PXABDOM', 'PXEXTREM', 'PXPERIPH', 'PXSKIN', 'PXMUSCUL', 'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt',
       'e_divatt_pt','e_memory_cg', 'e_lang_cg', 'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg']


categorical_features = np.in1d(features, ['MMONFLR', 'faq10', 'faq6', 'COPYSYM', 'COPYNUM', 'faq5', 'MMTREE', 'COPYHAND', 'moca_clock', 'MMSEASON', 'moca_letters', 
                                          'MMBALL', 'faq9', 'MMFLOOR', 'MMDRAW', 'MMMONTH', 'PXGENAPP', 'MMWATCH', 'CLOCKCIRC', 'faq8', 'MMHOSPIT', 'moca_naming', 
                                          'PXEXTREM', 'CLOCKNUM', 'PXMUSCUL', 'faq1', 'MMTREEDL', 'CLOCKTIME', 'PXABDOM', 'MMFLAG', 'COPYCIRC', 'MMAREA', 'faq3', 
                                          'moca_digits', 'CLOCKHAND', 'MMREAD', 'MMYEAR', 'MMREPEAT', 'moca_visuo_exec','MMHAND', 'MMBALLDL', 'PXCHEST', 'MMDATE', 
                                          'MMFLAGDL',  'MMSTATE','moca_repeat', 'MMFOLD', 'MMPENCIL', 'MMDAY',  'faq2', 'PXHEART', 'CLOCKSYM', 'faq4', 'moca_serial7', 
                                          'faq7', 'MMCITY', 'PXHEADEY', 'COPYTIME', 'PXPERIPH', 'PXSKIN', 'moca_fluency', 'moca_similarities', 'PXNECK', 'MMWRITE',
                                          'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
                                          'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg'])

data = pd.read_csv('../data/interim/data_adni.csv')
X = data[features]
y = data['CDGLOBAL']

In [21]:
X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)     

In [23]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(4)),repeats=list(int(x)*42+42 for x in range(5)), num_classes=3)
df.describe()

Wall time: 8.1 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.837500,0.924074,0.885204,0.880787,0.882292,0.816000,0.848469,0.849146,0.658333,0.983152,0.963265,0.820743,0.792708,0.907742,0.898980,0.945959
std,0.032997,0.018421,0.014383,0.015498,0.026200,0.028172,0.015180,0.015137,0.137596,0.007460,0.007693,0.066952,0.038840,0.008583,0.010120,0.008280
min,0.772727,0.888889,0.857143,0.849327,0.854167,0.770000,0.816327,0.817292,0.416667,0.972826,0.954082,0.705616,0.717803,0.889584,0.877551,0.937148
25%,0.823864,0.916667,0.876276,0.870633,0.864583,0.790000,0.840561,0.840885,0.562500,0.978261,0.959184,0.772418,0.761995,0.902599,0.893707,0.939717
50%,0.840909,0.921296,0.885204,0.881629,0.875000,0.815000,0.849490,0.849896,0.666667,0.983696,0.959184,0.822464,0.790404,0.908578,0.899660,0.944382
75%,0.855114,0.935185,0.894133,0.891309,0.888021,0.840000,0.858418,0.859792,0.750000,0.989130,0.965561,0.868207,0.827415,0.913660,0.905612,0.949393
max,0.886364,0.962963,0.913265,0.910774,0.947917,0.880000,0.867347,0.868958,0.833333,0.994565,0.979592,0.911232,0.857955,0.921149,0.911565,0.961860


## Random Search

In [24]:
from sklearn.model_selection import RandomizedSearchCV

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [26]:
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  9.1min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=0.5,
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                  

In [27]:
rf_search_rand.best_params_

{'n_estimators': 800,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 0.3,
 'max_depth': 20,
 'bootstrap': True}

In [28]:
best = rf_search_rand.best_estimator_
df_rand = splitrepeat_cv(X,y,best,splits=list(int(x)*42+42 for x in range(3)),repeats=list(int(x)*42+42 for x in range(3)), num_classes=3)
df_rand.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.819444,0.934156,0.882653,0.876800,0.896991,0.803333,0.849206,0.850162,0.685185,0.984903,0.966553,0.835044,0.800540,0.907464,0.899471,0.950098
std,0.036578,0.017595,0.017302,0.018624,0.032294,0.041533,0.023810,0.023656,0.165505,0.009326,0.009917,0.080731,0.051890,0.013455,0.015873,0.009654
min,0.772727,0.916667,0.857143,0.849327,0.864583,0.770000,0.816327,0.817292,0.416667,0.972826,0.959184,0.705616,0.724116,0.889584,0.877551,0.939390
25%,0.784091,0.925926,0.862245,0.855008,0.864583,0.780000,0.821429,0.822292,0.500000,0.972826,0.959184,0.747283,0.751894,0.892917,0.880952,0.941873
50%,0.818182,0.925926,0.887755,0.884470,0.885417,0.780000,0.857143,0.858750,0.750000,0.989130,0.959184,0.861413,0.799558,0.909423,0.904762,0.946739
75%,0.840909,0.953704,0.892857,0.885943,0.937500,0.840000,0.867347,0.867708,0.833333,0.989130,0.979592,0.911232,0.856692,0.918352,0.911565,0.962352
max,0.886364,0.962963,0.903061,0.901515,0.937500,0.880000,0.882653,0.882708,0.833333,0.994565,0.979592,0.911232,0.868371,0.928599,0.921769,0.962506


## Grid Search

In [29]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_estimators': [700, 800, 900],
               'max_features': [.2, .3, .4],
               'max_depth': [10, 20, 30],
               'min_samples_split': [8, 9, 10],
               'min_samples_leaf': [1, 2],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:   38.0s
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 972 out of 972 | elapsed:  4.8min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=0.5,
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=210,
                                    

In [30]:
rf_search_grid.best_params_

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 0.3,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 700}

In [31]:
best = rf_search_grid.best_estimator_
df_grid = splitrepeat_cv(X,y,best,splits=list(int(x)*42+42 for x in range(3)),repeats=list(int(x)*42+42 for x in range(3)), num_classes=3)
df_grid.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.816919,0.934156,0.881519,0.875538,0.898148,0.802222,0.849206,0.850185,0.694444,0.985507,0.967687,0.839976,0.803171,0.907295,0.899471,0.950356
std,0.035685,0.019883,0.018881,0.019918,0.035239,0.039299,0.025652,0.025585,0.150231,0.009798,0.010518,0.073564,0.046971,0.014201,0.017101,0.009532
min,0.772727,0.916667,0.852041,0.844697,0.854167,0.770000,0.811224,0.812083,0.500000,0.972826,0.959184,0.744565,0.751894,0.886498,0.874150,0.940282
25%,0.784091,0.916667,0.862245,0.855008,0.864583,0.780000,0.821429,0.822292,0.500000,0.972826,0.959184,0.747283,0.755366,0.892917,0.880952,0.942164
50%,0.818182,0.925926,0.887755,0.884470,0.895833,0.780000,0.857143,0.858750,0.750000,0.989130,0.964286,0.861413,0.795770,0.910698,0.904762,0.947191
75%,0.840909,0.953704,0.897959,0.890572,0.937500,0.840000,0.867347,0.867917,0.833333,0.994565,0.979592,0.911232,0.856692,0.918352,0.911565,0.961975
max,0.875000,0.962963,0.897959,0.895833,0.947917,0.870000,0.877551,0.877708,0.833333,0.994565,0.984694,0.913949,0.864583,0.925266,0.918367,0.962887


In [32]:
rf = RandomForestClassifier(n_estimators=700, max_features=.3, max_depth = 20, min_samples_split = 10, min_samples_leaf = 1, bootstrap=True, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(4)),repeats=list(int(x)*42+42 for x in range(5)), num_classes=3)
df.describe()

Wall time: 37.7 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.830682,0.932870,0.886990,0.881776,0.895312,0.811500,0.852551,0.853406,0.670833,0.984783,0.965561,0.827808,0.798943,0.909718,0.901701,0.948861
std,0.035916,0.017741,0.018078,0.019290,0.031062,0.036023,0.023521,0.023443,0.130997,0.008003,0.009346,0.064503,0.041808,0.013309,0.015680,0.008289
min,0.772727,0.916667,0.852041,0.844697,0.854167,0.770000,0.811224,0.812083,0.500000,0.972826,0.959184,0.744565,0.751894,0.886498,0.874150,0.939012
25%,0.809659,0.923611,0.881378,0.876315,0.872396,0.780000,0.840561,0.841198,0.583333,0.980978,0.959184,0.783514,0.772569,0.903320,0.893707,0.943735
50%,0.835227,0.925926,0.892857,0.888573,0.890625,0.805000,0.857143,0.858125,0.666667,0.986413,0.959184,0.825181,0.791351,0.910821,0.904762,0.946140
75%,0.855114,0.939815,0.897959,0.894781,0.903646,0.832500,0.868622,0.869167,0.770833,0.989130,0.971939,0.873868,0.813842,0.918867,0.912415,0.951972
max,0.886364,0.972222,0.908163,0.906145,0.958333,0.880000,0.887755,0.887917,0.833333,0.994565,0.984694,0.913949,0.871843,0.931685,0.925170,0.962887


In [3]:
# All samples
rf = RandomForestClassifier(n_estimators=700, max_features=.3, max_depth = 20, min_samples_split = 10, min_samples_leaf = 1, bootstrap=True, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(20)),repeats=list(int(x)*42+42 for x in range(20)), num_classes=3, imbalanced='over', avg_strategy='weighted', initial_split_seed=33433, initial_split_ratio=.25)
df.describe()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, cate

Wall time: 2h 20min 57s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity2,Specificity2,Accuracy2,AUC2,Sensitivity,Specificity,Accuracy,AUC
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,0.865673,0.926402,0.899549,0.896038,0.895697,0.843061,0.869379,0.869379,0.670147,0.988222,0.969830,0.829185,0.869379,0.865662,0.869379,0.951011
std,0.029818,0.022659,0.016458,0.016942,0.026727,0.028688,0.019969,0.019969,0.089292,0.006212,0.007219,0.044300,0.019969,0.020294,0.019969,0.008835
min,0.800000,0.871951,0.860544,0.854268,0.843537,0.795918,0.836735,0.836735,0.470588,0.974729,0.955782,0.729879,0.836735,0.830163,0.836735,0.937717
25%,0.838462,0.914634,0.887755,0.884897,0.877551,0.816327,0.853741,0.853741,0.588235,0.985560,0.962585,0.790508,0.853741,0.850352,0.853741,0.942480
50%,0.869231,0.926829,0.897959,0.893129,0.891156,0.850340,0.867347,0.867347,0.647059,0.989170,0.972789,0.820822,0.867347,0.862674,0.867347,0.950330
75%,0.892308,0.939024,0.911565,0.907833,0.911565,0.870748,0.884354,0.884354,0.705882,0.992780,0.976190,0.849331,0.884354,0.881012,0.884354,0.955615
max,0.930769,0.969512,0.935374,0.931707,0.959184,0.904762,0.918367,0.918367,0.882353,1.000000,0.989796,0.935761,0.918367,0.915620,0.918367,0.969046


# Outputs

In [4]:
### Save outputs ###
q = 'CDR_MultiC_AllFeatures_400'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)